In [1]:
from google.colab import drive
drive.mount('/content/drive/')
DIR='/content/drive/MyDrive/Research/OntoConnectWithGNN/GNN_1/Anatomy/'

import os
os.chdir(DIR)

Mounted at /content/drive/


In [2]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

!pip install git+https://github.com/facebookresearch/fastText.git
!pip install git+https://github.com/facebookresearch/PyTorch-BigGraph.git

<IPython.core.display.Javascript object>

1.8.1+cu101
10.1
  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-hn4n94db
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-hn4n94db
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3090360 sha256=2f4ab24473cdcee339cd2d42ec0981ee1a09a46518856c45d94ab0f14eec257a
  Stored in directory: /tmp/pip-ephem-wheel-cache-6gj3_mdz/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext
  Cloning https://github.com/facebookresearch/PyTorch-BigGraph.git to /tmp/pip-req-build-o3cxi0oc
  Running command git clone -q https://github.com/facebookresearch/PyTorch-BigGraph.git /tmp/pip-req-build-o3cxi0oc
  Created wheel for torchbiggraph: filename=torchbiggraph-1.0.1.dev0-cp37-none-any.whl size=119004 sha256=1992d79b8f3f981e885f38f161b4f52619e9bbeb6df6166c6e37b809ec96c692
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbxa_ur7/wheels/d5/d0/e8/9fa5

In [3]:
#from OntoSimImports import *
%run OntoSimImports.py

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
#import OntoSimConstants
%run OntoSimConstants.py
from OntoSimConstants import *

In [5]:
def assignVar():
    conf = {
        'src_fl_nm': DATA_DIR+'gnnentity/source_gnn_meta.json',
        'trgt_fl_nm': DATA_DIR+'gnnentity/target_gnn_meta.json',
        'op_fl_nm': DATA_DIR+'output/meta_sim/meta_sim'
    }
    return conf

In [6]:
def getDataParam():
    data_param_fl_nm = code_path + data_param_json
    with open(data_param_fl_nm) as f:
        data_param = json.load(f)
    return data_param

In [7]:
def makeDir():
  fl_path=DATA_DIR+'output/meta_sim'
  os.makedirs(fl_path, exist_ok=True)

In [8]:
def loadDict(conf):
    with open(code_path+conf['dict_fl_nm']) as f:
        embed_json_data = json.load(f)

    return embed_json_data

In [9]:
def loadSrcTrgt(conf):
    with open(code_path+conf['src_fl_nm']) as f:
        src_data = json.load(f)

    with open(code_path+conf['trgt_fl_nm']) as f:
        trgt_data = json.load(f)

    return src_data, trgt_data

In [10]:
def populateSrcNpVec(src_data):
    src_vec = []
    src_key_arr = []
    for src_key in src_data.keys():
        src_key_arr.append(src_key)
        src_vec.append(src_data[src_key]["vector"])

    src_vec = np.asarray(src_vec)

    return src_key_arr, src_vec

In [11]:
def cos_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    cos-dist = 1 - cos(@) // @ is angle between the two vectors
    @ = 0, cost-dist = 1 - 1 = 0
    @ = 90, cost-dist = 1 - 0 = 1
    @ = 180, cost-dist = 1 - (-1) = 2
    @ = 270, cost-dist = 1 - 0 = 1
    @ = 360, cost-dist = 1 - 1 = 0
    more the distance less the similarity
    """
    # print(type(matrix))
    # print(matrix.shape) #(3306, 300)
    # print(type(vector))
    # print(vector.shape) #(1, 300)
    return scipy.spatial.distance.cdist(matrix, vector, 'cosine').reshape(-1)

In [12]:
def wordSim_Cosine(target_data, src_np_vec , source_keys):

  op_copy_cosine = OrderedDict()

  for trgt_key in target_data:  # For each target data
    op_copy_cosine_tmp = OrderedDict()
    trgt_vec = target_data[trgt_key]["vector"]
    trgt_vec = np.asarray(trgt_vec)
    trgt_vec = trgt_vec.reshape(1, trgt_vec.shape[0]) #making (300,) to (1,300)
    result = cos_cdist(src_np_vec, trgt_vec) #cosine distances between matrix(src), vector(target)
    # get the indices of the five(k) smallest values, # Indices not sorted
    tmp_inds = np.argpartition(result, top_k)[:top_k]
    ## Indices sorted by value from smallest to largest
    top_inds = tmp_inds[np.argsort(result[tmp_inds])]

    for top_ind in top_inds:
      src_key = source_keys[top_ind]
      cosine_val = result[top_ind]

      # print(1 - 0.9999999999999997)
      # print(1 - 0.9999999999999998)
      # print(1 - 0.9999999999999999)
      cosine_lst = [1.1102230246251565e-16, 2.220446049250313e-16, 3.3306690738754696e-16]
      if(cosine_val in cosine_lst):
          cosine_val = 0.0

      op_copy_cosine_tmp[src_key] = cosine_val

    op_copy_cosine[trgt_key] = op_copy_cosine_tmp

  return op_copy_cosine

In [13]:
def wordSim_Euclidean(target_data, src_np_vec , source_keys):

  op_copy_euclidean = OrderedDict()

  for trgt_key in target_data:  # For each target data
    op_copy_euclidean_tmp = OrderedDict()
    trgt_vec = target_data[trgt_key]["vector"]
    trgt_vec = np.asarray(trgt_vec)
    trgt_vec = trgt_vec.reshape(1, trgt_vec.shape[0]) #making (300,) to (1,300)
    result = ec_cdist(src_np_vec, trgt_vec) #euclidean distances between matrix(src), vector(target)
    # get the indices of the five(k) smallest values, # Indices not sorted
    tmp_inds = np.argpartition(result, top_k)[:top_k]
    ## Indices sorted by value from smallest to largest
    top_inds = tmp_inds[np.argsort(result[tmp_inds])]

    for top_ind in top_inds:
      src_key = source_keys[top_ind]
      euclidean_val = result[top_ind]
      op_copy_euclidean_tmp[src_key] = euclidean_val

    op_copy_euclidean[trgt_key] = op_copy_euclidean_tmp

  return op_copy_euclidean

In [14]:
def ec_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    more the distance less the similarity
    """
    # print(type(matrix))
    # print(matrix.shape) #(3306, 300)
    # print(type(vector))
    # print(vector.shape) #(1, 300)
    return scipy.spatial.distance.cdist(matrix, vector, 'euclidean').reshape(-1)

In [15]:
def saveOp(op_meta, op_fl_nm):
    with open(code_path+op_fl_nm, 'w') as outfile:
        json.dump(op_meta, outfile, indent=4)

In [16]:
#################### Main Code START ####################
def genMetaSim(word_sim_ind):
    try:
        print("#################### GenMetaSim START ####################")
        strt_tm = datetime.datetime.now()
        conf = assignVar()
        makeDir()
        if (word_sim_ind == word_sim_ind_1): ### cosine
            src_data, trgt_data = loadSrcTrgt(conf)
            source_keys, src_np_vec = populateSrcNpVec(src_data)
            op_copy_cosine = wordSim_Cosine(trgt_data, src_np_vec, source_keys)
            saveOp(op_copy_cosine, conf["op_fl_nm"] + '_cosine.json')
        elif (word_sim_ind == word_sim_ind_2): ### euclidean
            src_data, trgt_data = loadSrcTrgt(conf)
            source_keys, src_np_vec = populateSrcNpVec(src_data)
            op_copy_euclidean = wordSim_Euclidean(trgt_data, src_np_vec, source_keys)
            saveOp(op_copy_euclidean, conf["op_fl_nm"] + '_euclidean.json')

        time.sleep(wait_time)
    except Exception as exp:
        raise exp
    finally:
        end_tm = datetime.datetime.now()
        total_time_taken = end_tm - strt_tm
        print("Total time taken :- " + str(total_time_taken))
        print("#################### GenMetaSim FINISH ####################")

#################### Main Code END ####################

In [17]:
####
if __name__=="__main__":
  data_param = getDataParam()
  data_param = data_param["db"]
  genMetaSim(data_param["sim_ind"])

#################### GenMetaSim START ####################
Total time taken :- 0:00:30.195491
#################### GenMetaSim FINISH ####################
